In [1]:
import pandas as pd
import numpy as np

In [2]:
from utils.freebase import *
from utils.data_initial import *
from utils.data_generated import *

# Ethnicity values

In [3]:
df = load_cmu_character_metadata()

In [4]:
etnicities = list(df.actor_ethnicity.dropna().unique())

In [5]:
# query the values associated to the ethnicity freebase ids contained in our data
mappings = {}
not_found = []
for etn in etnicities:
    val = query_freebase_id_from_wikidata(etn)
    if val == None:
        not_found.append(etn)
    else:
        mappings[etn] = val

weird result for /m/019kn7
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q161652'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Japanese people'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31340083'}, 'sLabel': {'type': 'literal', 'value': 'Q31340083'}}]

weird result for /m/0j6x8
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q170355'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Indigenous Australians'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12060728'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aboriginal Australians'}}]

weird result for /m/062_25
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1065371'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Italian Brazilians'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q54864438'}, 'sLabel': {'type': 'literal', 'value': 'Q54864438'}}]

weird result for /m/0640_7

In [6]:
# manually correct the mappings that were under an unexpected format
mappings['/m/019kn7'] = 'Japanese'
mappings['/m/0j6x8'] = 'Aboriginal Australians'
mappings['/m/062_25'] = 'Italian Brazilians'
mappings['/m/0640_7q'] = 'Moroccan Jews'
mappings['/m/0180zw'] = 'Kikuyu'
mappings['/m/09snp5'] = 'Muhajir'
mappings['/m/03x1x'] = 'Haudenosaunee Confederacy'
mappings['/m/06bkf'] = 'Quebeckers'

In [7]:
# create dataframe
# set mappings for ids without values to NaN
mappings.update({id: np.NaN for id in not_found})

df = pd.DataFrame(data=mappings.items(), index=range(0,len(mappings)), columns=['freebase_id', 'ethnicity'])

In [8]:
# save dataframe
df.to_pickle(PATH_DATA_GEN + FILENAME_ETHNICITIES)

<a id='combine_cmu_imdb_movie'></a>
# Combine CMU with IMDB

In [9]:
df_imdb = load_imdb_title_basics()

In [10]:
df_movie_md = load_cmu_movie_metadata()

In [11]:
# keep only the timespan from 1996 to 2011
# see milestone_2.ipynb for the reasoning
df_movie_md = df_movie_md[df_movie_md.release_date.apply(lambda d: d.year) >= 1996]
df_movie_md = df_movie_md[df_movie_md.release_date.apply(lambda d: d.year) <= 2011]

In [12]:
# drop original title to avoid double entries
df_imdb.drop('original_title', axis='columns', inplace=True)

In [13]:
# we merge on the title, doing a left outer join
merged = df_movie_md.merge(df_imdb, left_on='movie_name', right_on='primary_title', how='left')

In [14]:
# keep only movies
merged = merged[merged.type == 'movie']

In [15]:
# since titles are not unique, keep only those with matching release date and runtime
merged = merged[merged.release_date.apply(lambda r: r.year) == merged.start_year]

In [16]:
# we see that there are still duplicate entries
duplicates = merged.movie_id_freebase.value_counts()
duplicates = duplicates[duplicates > 1]
duplicates

/m/0h1z21s    5
/m/09vq1kn    4
/m/064p159    4
/m/0j9nm_x    3
/m/0gy0m6v    3
             ..
/m/04n043k    2
/m/02pnrmw    2
/m/0gg5jd     2
/m/0805fwx    2
/m/0242q0     2
Name: movie_id_freebase, Length: 271, dtype: Int64

In [17]:
df_imdb_rating = load_imdb_title_rating()

In [18]:
# keep the ones with the most imdb votes, assuming that those entries are those with the most information
df = merged[merged.movie_id_freebase.isin(list(duplicates.index))].merge(df_imdb_rating, on='title_id', how='left')

In [19]:
to_drop = []
for id in list(duplicates.index):
    dft = df[df.movie_id_freebase == id]
    ids = list(dft.sort_values('num_votes', ascending=False).iloc[1:].title_id.values)
    to_drop += ids

In [20]:
merged = merged[~merged.title_id.isin(to_drop)]

In [21]:
# we see that there are no duplicates left
duplicates = merged.movie_id_freebase.value_counts()
duplicates = duplicates[duplicates > 1]
duplicates

Series([], Name: movie_id_freebase, dtype: Int64)

In [22]:
# use imdb runtime information for missing ones in the cmu dataset
merged.runtime = merged.runtime.fillna(merged.runtime_minutes)
merged.runtime = merged.runtime.astype(pd.Float32Dtype())

In [23]:
# drop columns we don't need
# we keep both genre columns separate, because their granularity is very different (as we saw in our eda) and this could be useful
merged = merged.drop(['start_year', 'end_year', 'type', 'primary_title', 'runtime_minutes'], axis='columns')

In [24]:
# reorder columns
merged = merged.iloc[:, [0,1,9,2,3,4,5,6,7,10,8,11]]

In [25]:
# rename columns
merged = merged.rename({'title_id': 'title_id_imdb', 'genres_x': 'genres_cmu', 'genres_y':'genres_imdb'}, axis='columns')

In [26]:
merged = merged.reset_index()

In [27]:
# save dataframe
merged.to_pickle(PATH_DATA_GEN + FILENAME_MOVIE_METADATA)

# Process the other datasets

Since we are interested in connections between people, we do not need everything in our datasets. Hence we only consider the parts of the datasets which are relevant to us.

In [28]:
movie_metadata = load_movie_metadata()

In [29]:
movie_ids = movie_metadata.title_id_imdb.unique()

### IMDb movie crew data

In [30]:
crew_data = load_imdb_title_crew()

In [31]:
# keep only information for movies which are in our processed dataset
crew_data = crew_data[crew_data.title_id.isin(movie_ids)]

In [32]:
crew_data = crew_data.reset_index()

In [33]:
# save dataframe
crew_data.to_pickle(PATH_DATA_GEN + FILENAME_MOVIE_CREW)

### IMDb movie principals data

In [34]:
# beware that this dataframe is large, with 53001047 rows
principals_data = load_imdb_title_principals()

In [35]:
# keep only information for movies which are in our processed dataset
principals_data = principals_data[principals_data.title_id.isin(movie_ids)]

In [36]:
principals_data = principals_data.reset_index()

In [37]:
# save dataframe
principals_data.to_pickle(PATH_DATA_GEN + FILENAME_MOVIE_PRINCIPALS)

### IMDb movie ratings data

In [38]:
ratings_data = load_imdb_title_rating()

In [39]:
# keep only information for movies which are in our processed dataset
ratings_data = ratings_data[ratings_data.title_id.isin(movie_ids)]

In [40]:
ratings_data = ratings_data.reset_index()

In [41]:
# save dataframe
ratings_data.to_pickle(PATH_DATA_GEN + FILENAME_MOVIE_RATINGS)

<a id='combine_cmu_imdb_people'></a>
### People data

In [42]:
char_data = load_cmu_character_metadata()

In [43]:
people_data = load_imdb_name_basics()

In [44]:
def aggregate_ids(ids, acc):
    for id in ids:
        acc.append(id)

In [45]:
people_ids = []

for ids in crew_data.directors.dropna():
    aggregate_ids(ids, people_ids)

for ids in crew_data.writers.dropna():
    aggregate_ids(ids, people_ids)

for id in principals_data.persone_name_id:
    people_ids.append(id)

In [46]:
# keep only information for people which are in our processed datasets
people_data = people_data[people_data.person_name_id.isin(people_ids)]

In [47]:
# we only care about the actor, not the character that is portrayed
actor_data = char_data[['actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name']]
actor_data = actor_data.groupby('actor_name').agg(list)

In [48]:
# combine data from imdb and from cmu
# we combine on name, which is not ideal because of potential typos, conventions etc, but is the best we have
merged = people_data.merge(actor_data, left_on='person_name', right_on='actor_name', how='left')

In [49]:
def keep_max(attr):
    if type(attr) == str or type(attr) == float:
        return attr

    # not very elegant nor efficient to create a dataframe each time
    # but NaN values are a pain and this way at least it works right
    df = pd.DataFrame(attr)
    df = df.dropna().value_counts().sort_values()
    if len(df) == 0:
        return np.NaN
    else:
        return df.index[0][0]

In [50]:
# keep values that occur most often
merged.actor_gender = merged.actor_gender.map(keep_max, na_action='ignore')
merged.actor_height = merged.actor_height.map(keep_max, na_action='ignore').astype(pd.StringDtype())
merged.actor_ethnicity = merged.actor_ethnicity.map(keep_max, na_action='ignore').astype(pd.StringDtype())

In [51]:
merged = merged.rename(columns={'actor_gender': 'gender', 'actor_height': 'heigth', 'actor_ethnicity': 'ethnicity'})

In [52]:
merged = merged.reset_index()

In [53]:
# save dataframe
merged.to_pickle(PATH_DATA_GEN + FILENAME_PEOPLE)

### Aggregate movies per person

In [3]:
people = load_people()

In [4]:
principals = load_movie_principals()

In [5]:
crew = load_movie_crew()

In [6]:
movies = load_movie_metadata()

In [7]:
# get every profession in our dataset
professions = pd.DataFrame([profession for sublist in people.primary_profession.dropna() for profession in sublist], columns=['profession']).profession.unique()

In [8]:
# build a datafram where we aggregate the roles and movies of each person in our data
is_in_movies = pd.DataFrame(index=people.person_name_id.values, columns=list(professions))

In [9]:
def append_to_cell(df, x, y, value):
    '''
    Appends value to the cell df[x,y].
    '''
    if not type(df.at[x,y]) == list and np.isnan(df.at[x,y]):
        df.at[x,y] = [value]
    else:
        df.at[x,y].append(value)

In [10]:
# keep track of how many values cannot be added due to key errors
i = 0

# add people from movie.principals
for title_id, group in principals.groupby('title_id'):
    for _, row in group.iterrows():
        try:
            append_to_cell(is_in_movies, row.persone_name_id, row.category, title_id)
        except:
            i += 1

In [11]:
# add people from movie.crew
for _, row in crew.iterrows():
    if type(row.directors) == list:
        for director in row.directors:
            try:
                append_to_cell(is_in_movies, director, 'director', row.title_id)
            except:
                i += 1
    if type(row.writers) == list:
        for writer in row.writers:
            try:
                append_to_cell(is_in_movies, writer, 'writer', row.title_id)
            except:
                i += 1

In [12]:
print("We could not put {:d} entries into our dataframe.".format(i))

We could not put 4103 entries into our dataframe.


In [13]:
# save dataframe
is_in_movies.to_pickle(PATH_DATA_GEN + FILENAME_IS_IN_MOVIES)